<a href="https://colab.research.google.com/github/IldarAltynbaev/MLSnippetSimpleNeuroNwtwork/blob/TestingColab/TaskFromStepik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [168]:
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)
   # print(conv2d_out.shape)
    #print(conv2d_out)
    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


In [182]:
class Conv2dLoop(ABCConv2d):
    def __call__(self, input_tensor):
        output_height = (input_tensor.shape[2] + 2 * 0 - (self.kernel_size - 1) - 1) // self.stride + 1

        output_width = (input_tensor.shape[3] + 2 * 0 - (self.kernel_size - 1) - 1) // self.stride + 1
        output_tensor = torch.zeros(input_tensor.shape[0],self.kernel.shape[0], output_height, output_width )
        print(output_tensor)

        for batch_number in range(input_tensor.shape[0]):
          current_dot_tensor = torch.zeros(self.kernel.shape[1], output_height, output_width)
          for each_kernel in range(self.kernel.shape[0]):

            for each_kernel_layer in range(self.kernel.shape[1]):
              current_channel = input_tensor[batch_number, each_kernel_layer]
              i = 0
              j = 0

              for each_hight in range(0, input_tensor.shape[2], self.stride):
                if (each_hight + self.kernel_size) < input_tensor.shape[2]:
                  j = 0
                  ++i
                  for each_wight in range(0, input_tensor.shape[3], self.stride):
                    if (each_wight+self.kernel_size) < input_tensor.shape[3]:
                      high = each_hight + self.kernel_size
                      width = each_wight+self.kernel_size
                      high = int(high)
                      width = int(width)

                      #print(current_channel[each_hight:high, each_wight:width])
                      #print(self.kernel[each_kernel, each_kernel_layer])
                      dot_product = current_channel[each_hight:high, each_wight:width]*self.kernel[each_kernel, each_kernel_layer]
                      #print(int(dot_product.sum()))
                      current_dot_tensor[each_kernel_layer, i, j] = dot_product.sum()
                      #output_tensor[batch_number, self.kernel.shape[each_kernel], i, j] = dot_product.sum()
                      ++j
          #print(current_dot_tensor.shape)
          #print(current_dot_tensor)
          current_dot_tensor = torch.sum(current_dot_tensor, dim = 0, keepdim=False)
          #print(current_dot_tensor.shape)
          for col in range(output_height):
            for row in range(output_width):
              output_tensor[batch_number, each_kernel, col, row] = current_dot_tensor[i,j]
              #print(batch_number, each_kernel, col, row)
              #print(current_dot_tensor[i,j])
          #output_tensor[batch_number, each_kernel, i1, j1] = current_dot_tensor.reshape(-1).sum()
          #batch_sum = sum(batch_sum)
          #print(output_tensor)

        return output_tensor

print(test_conv2d_layer(Conv2dLoop))


tensor([[[[0.]]],


        [[[0.]]]])
True
